In [42]:
import numpy as np
import os
import sys
import time
import matplotlib.pyplot as plt
import nltk  # do not forget to nltk.download
%matplotlib inline

In [2]:
!mkdir data

In [3]:
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip -P data

--2018-12-18 12:14:09--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.40.194
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.40.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4475746 (4.3M) [application/zip]
Saving to: ‘data/wikitext-2-v1.zip’

wikitext-2-v1.zip   100%[===================>]   4.27M  7.49MB/s    in 0.6s    

2018-12-18 12:14:10 (7.49 MB/s) - ‘data/wikitext-2-v1.zip’ saved [4475746/4475746]



In [9]:
!unzip data/wikitext-2-v1.zip -d data

Archive:  data/wikitext-2-v1.zip
   creating: data/wikitext-2/
  inflating: data/wikitext-2/wiki.test.tokens  
  inflating: data/wikitext-2/wiki.valid.tokens  
  inflating: data/wikitext-2/wiki.train.tokens  


In [17]:
!head data/wikitext-2/wiki.train.tokens -n 15

 
 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more <unk> for series newcomers . Char

In [43]:
from collections import Counter

SOS_WORD = '<s>'
EOS_WORD = '<EOS>'
UNK_WORD = '<unk>'

def create_vocabe(fname, max_size=1e6):
    # TODO: read about wikitext - how many words are typically remained, lowercased? = title = ?
    word_counter = Counter()
    word_counter[EOS_WORD] = 1e16
    word_counter[SOS_WORD] = 1e15
    with open(fname, 'r', encoding='utf-8-sig') as f:
        for line in f:
            if line.strip():
                words = map(lambda w: w.lower(), line.strip().split())
                word_counter.update(words)
    vocabe = [w for (w, count) in word_counter.most_common(int(max_size))]
    return vocabe

vocabe = create_vocabe('data/wikitext-2/wiki.train.tokens')
word_id = dict((w, i) for i, w in enumerate(vocabe))
print("words in vocabe:", len(vocabe))

UNK = word_id[UNK_WORD]
SOS = word_id[SOS_WORD]  # как этого называть? забыл
EOS = word_id[EOS_WORD]

def word2id(w):
    return word_id[w] if w in word_id else UNK

def id2word(_id):
    return vocabe[i]

def encode_sent(sentence):
    return [word2id(w) for w in sentence.strip().lower().split()]

def decode_sent(tokens):
    return ' '.join(id2word(t) for t in tokens)

def encode_text(fname, min_sentence_len=5):
    all_sentences = dict()  # number of tokens -> list of sentences
    with open(fname, 'r', encoding='utf-8-sig') as f:
        for line in f:
            if line.strip():
                if np.random.normal() > 2:
                    print(nltk.sent_tokenize(line))
                    print()
                sents = line.split('.')  # without '.' ! # todo - !,?
                for sent in sents:
                    tokens = encode_sent(sent) + [word2id('.')]
                    n_tokens = len(tokens)
                    if n_tokens < min_sentence_len or sent.strip()[0] == '=':
                        continue
                        
                    if n_tokens not in all_sentences:
                        all_sentences[n_tokens] = [sent]
                    else:
                        all_sentences[n_tokens].append(sent)
    return all_sentences

train_sentences = encode_text('data/wikitext-2/wiki.train.tokens')
valid_sentences = encode_text('data/wikitext-2/wiki.valid.tokens')
test_sentences = encode_text('data/wikitext-2/wiki.test.tokens')

words in vocabe: 28913
[' The Little Rock Arsenal was classified in 1860 as an " arsenal of deposit , " meaning that it was simply a warehouse for the storage of weapons intended for the use of the state militia in times of crisis .', 'Thus there were no substantial operations for ordnance fabrication or repairs , nor for the manufacture of cartridges at the time the Arsenal fell into State hands .', 'Most of these operations were started from scratch through the efforts of the Arkansas Military Board .']

[' The " <unk> of Work done at Little Rock Arsenal , <unk> " continue at about the same pace and scale from August 1862 until August 1863 .', '<unk> to the " Summary " for August , 1863 is the ominous <unk> , " During the last week in the month , nearly all stores at the Arsenal have been packed and sent to Arkadelphia , in obedience to orders from Chief of Ordnance , District of Arkansas . "', "This then marks the beginning of the evacuation of ordnance activities from Little Rock ,

[' = = = Ratings = = =']

[' = = Aftermath = =']

[' The corn crake breeds from Britain and Ireland east through Europe to central Siberia .', 'Although it has vanished from much of its historic range , this bird was once found in suitable habitats in Eurasia everywhere between latitudes 41 ° N and 62 ° N. There is also a sizable population in western China , but this species nests only rarely in northern Spain and in Turkey .', 'Old claims of breeding in South Africa are incorrect , and result from <unk> of eggs in a museum collection which are actually those of the African rail .']

[' During the 1920s and 1930s , the city re @-@ emerged as a major tourist destination .', 'Under the influence of Sam Maceo and <unk> Maceo , the city exploited the prohibition of liquor and gambling in clubs like the <unk> Room , which offered entertainment to wealthy <unk> and other out @-@ of @-@ <unk> .', 'Combined with prostitution , which had existed in the city since the Civil War , Galveston beca

[" Garth Ennis ' run is also much loved , particularly Dangerous Habits , which was voted the best Garth Ennis story on Comic Book Resources , ahead of his work on Preacher , The Boys and The <unk> .", "In the same article Brian <unk> describes John 's one time love , Kit Ryan , as one of the series ' most memorable characters .", "The popularity of Ennis and Dillon 's run on Hellblazer is also credited for Vertigo agreeing to publish their seminal series Preacher .", 'Empire Magazine called Dangerous Habits storyline as " rightly one of the most celebrated in comic book history . "', 'Glenn <unk> who was the cover artist during Ennis \' run , won an <unk> Ward for " Best Cover Artist " in 1995 .']

[' Fantasy fiction author John Shirley is credited in making three Hellblazer novels , including the <unk> of the Constantine film .', 'The novel Hellblazer : War Lord features Constantine talking about " another John Constantine in an alternate universe , [ who ] has black hair and lives m

[' The release for the film was originally planned for 2007 , then 2008 and 2009 , before its 2010 release .', '<unk> of the film were shown at Wikimania 2007 in <unk> prior to its completion .', 'Editors in the audience had mixed views on the film .', 'In 2008 , footage from the film was used in an official fundraising video by the Wikimedia Foundation .', 'The film had its premiere at Wikimania 2010 in <unk> in July 2010 , before an audience of approximately 300 people .', 'A trailer for the movie was released in October 2010 .']

[' Super Mario Land is remembered for its <unk> Super Mario elements and " twist on just about every Mario mainstay <unk> " .', 'Many of its new elements did not recur later in the series , making Super Mario Land strange compared to the rest of the series , or what IGN \'s Thomas described as a " singular <unk> " .', 'IGN \'s Marc Nix felt <unk> that Super Mario Land was the only <unk> Mario game , with " <unk> <unk> of white " and <unk> instead of the " s

[' Japanese - <unk>']

[' Following this defeat , the Islamic Courts Union splintered into several different factions .', "Some of the more radical elements , including Al @-@ Shabaab , regrouped to continue their insurgency against the TFG and oppose the Ethiopian military 's presence in Somalia .", 'Throughout 2007 and 2008 , Al @-@ Shabaab scored military victories , seizing control of key towns and ports in both central and southern Somalia .', 'At the end of 2008 , the group had captured <unk> but not Mogadishu .', "By January 2009 , Al @-@ Shabaab and other militias had managed to force the Ethiopian troops to retreat , leaving behind an under @-@ equipped African Union peacekeeping force to assist the Transitional Federal Government 's troops ."]

[" The Federal Government of Somalia was established on 20 August 2012 , concurrent with the end of the TFG 's interim mandate .", 'It represents the first permanent central government in the country since the start of the civil war .'

...


[' = = = Post Vietnam era = = =']

[' = = Medal of Honor recipients = =']

[' = = <unk> of the Marine Corps = =']

[' = = = Ratings = = =']

[' The information from the body leads Section 9 to uncover the mysterious Project 2501 .', 'Section 6 claims the project was created to catch the hacker , but it was initiated before his appearance .', 'Section 9 speculates that the project itself created the Puppet Master , who then escaped , and Section 6 now wants him back .', '<unk> <unk> , head of Section 9 , suspects that the project and the Puppet Master are tools of the Ministry of Foreign Affairs .', 'The escape might lead to the release of secrets that could embarrass Section 6 and the Ministry .']

[' The opening credits of the film were produced by the CG director , <unk> Tanaka .', 'Tanaka converted code in a computer language displayed in <unk> Japanese letters to numbers before inserting them into the computer to generate the credits .', "The origin of this code is the names of th

[' Digital download']

[" Hasted 's arguments were largely accepted by influential historian Robert Chambers , and the story was generally treated as a folk myth .", 'A letter to the British Medical Journal in 1869 pointed out that <unk> were not in use in Kent in the 12th century , and that in older styles of English handwriting the 1 and 5 characters could easily be confused , and suggested a correct <unk> of 1500 .', 'The Biddenden Maids were occasionally mentioned in pieces on conjoined twins , particularly after Chang and <unk> Bunker proved that conjoined twins could live to an advanced age and lead relatively normal lives .', 'Notes and <unk> magazine called in 1866 for a close examination of Biddenden documents , the editors describing Hasted \'s conclusions as " very obscure and unsatisfactory " and questioning why the names " Eliza and Mary Chulkhurst " should have been added to the design of cakes granted by a family named Preston , but no significant research into the tradi

[' = = = Frankfurt = = =']

[' = = = <unk> = = =']

[" The construction of Canning Dam ended a long period during which Perth 's water supply was generally unsatisfactory in quality ( either due to salinity or bacterial pollution or both ) and in quantity .", 'The project , the biggest public works program of the decade , stimulated significant growth in the local economy and provided desperately needed work for around five hundred men .']

[' = = History = =']

[' = = Release and reception = =']

[' The magnetic field of a star is generated within regions of the interior where convective circulation occurs .', 'This movement of <unk> plasma functions like a <unk> , wherein the movement of <unk> charges induce magnetic fields , as does a mechanical <unk> .', 'Those magnetic fields have a great range that extend throughout and beyond the star .', "The strength of the magnetic field varies with the mass and composition of the star , and the amount of magnetic surface activity depends upo

[' Before 1983 , US 2 used a different routing through <unk> and crossed the <unk> River on what is nicknamed the " <unk> Bridge " .', 'Built as a part of a <unk> <unk> on the river , the water level is actually higher than the road surface .', 'This produces a <unk> effect , giving the bridge its nickname .', 'The <unk> <unk> and Paper Company was organized in 1916 and needed a dam on the <unk> River to supply their mill .', 'This dam would require a large section of the city to be flooded , and shallow river banks meant difficulties in any bridge construction .', 'Instead of expensive <unk> , a concrete tank was built <unk> in the river bed ; the sides of this tank provided man @-@ made banks higher than the natural banks .', 'The Michigan Works Progress Administration described the bridge as having " concrete bulkheads , formed by the side spans of the bridge , [ that ] allow the mill to maintain the water level several feet above the roadbed . "', 'The <unk> Tourism Council stated 

[" Books 61 – 63 of <unk> 's Roman History describe the reign of <unk> .", 'Only fragments of these books remain and what does remain was abridged and altered by John <unk> , an 11th @-@ century monk .']

[' = = Arts , culture and religion = =']

[' Although both <unk> operated openly , neither was in control of Free Derry .', "The barricades were manned by unarmed ' <unk> ' .", 'Crime was dealt with by a volunteer force called the Free Derry Police , which was headed by Tony O <unk> , a Derry footballer and Northern Ireland International .']

[' = = <unk> = =']

[' = = = Courts of law and legal enforcement = = =']

[' Preparations to set up the Croatian National Guard ( <unk> <unk> garde – <unk> ) began on 12 April 1991 .', 'Establishment of the <unk> as a police force with military capabilities was thought necessary by the Croatian authorities following armed clashes in <unk> and at <unk> Lakes in March and due to the possibility of further confrontation with the <unk> .', "The <unk>

In [38]:
train_sentences[8]

['69 cal smoothbore ( percussion ) 357 \n',
 ' <unk> <unk> cal rifle @-@ <unk> 900 \n',
 ' Guard , office , and police duties ',
 ' That site later became Fort Logan H',
 ' <unk> , and the Works Progress Administration ',
 ' Her ashes were scattered in Storrington churchyard ',
 ' Along with Greenaway , illustrator Alice B',
 " Edmund 's , Pitlake , in 1962 ",
 ' <unk> Children of the Allies ; J',
 ' Beautiful Bible Pictures ; Blackie , 1932 \n',
 ' Simon the Swan ; Blackie , 1988 \n',
 ' Andrews , Croydon , 1948 , 1962 \n',
 ' That game was not FIFA @-@ recognised ',
 ' The holotype is a male 21 @',
 ' There are five pairs of gill slits ',
 ' Richards had previously coached the Minnesota Wild ',
 ' Charts were printed on silver bromide paper ',
 ' An accompanying music video highlights social <unk> ',
 ' He was assisted by Dana Jon <unk> ',
 ' No significant changes have been made since ',
 '@ 59 m ) above the waterline ',
 ' The equivalent statistics for the 18 @',
 '@ 5 @-@ caliber 

In [1]:
import warnings
warnings.filterwarnings('ignore')

import glob
import time
import math

import mxnet as mx
from mxnet import gluon, autograd
from mxnet.gluon.utils import download

import gluonnlp as nlp
import numpy as np

In [2]:
num_gpus = 1
context = [mx.gpu(i) for i in range(num_gpus)] if num_gpus else [mx.cpu()]
log_interval = 200

In [3]:
batch_size = 20 * len(context)
lr = 20
epochs = 3
bptt = 35
grad_clip = 0.25

In [4]:
dataset_name = 'wikitext-2'
train_dataset, val_dataset, test_dataset = [
    nlp.data.WikiText2(
        segment=segment, bos=None, eos='<eos>', skip_empty=False)
    for segment in ['train', 'val', 'test']
]

vocab = nlp.Vocab(
    nlp.data.Counter(train_dataset), padding_token=None, bos_token=None)

bptt_batchify = nlp.data.batchify.CorpusBPTTBatchify(
    vocab, bptt, batch_size, last_batch='discard')
train_data, val_data, test_data = [
    bptt_batchify(x) for x in [train_dataset, val_dataset, test_dataset]
]

In [5]:
model_name = 'standard_lstm_lm_200'
model, vocab = nlp.model.get_model(model_name, vocab=vocab, dataset_name=dataset_name, pretrained=True)
print(model)
print(vocab)

model.initialize(mx.init.Xavier(), ctx=context)

trainer = gluon.Trainer(model.collect_params(), 'sgd', {
    'learning_rate': lr,
    'momentum': 0,
    'wd': 0
})
loss = gluon.loss.SoftmaxCrossEntropyLoss()

Vocab file is not found. Downloading.
StandardRNN(
  (embedding): HybridSequential(
    (0): Embedding(33278 -> 200, float32)
    (1): Dropout(p = 0.2, axes=())
  )
  (encoder): LSTM(200 -> 200, TNC, num_layers=2, dropout=0.2)
  (decoder): HybridSequential(
    (0): Dense(200 -> 33278, linear)
  )
)
Vocab(size=33278, unk="<unk>", reserved="['<eos>']")


In [6]:
params = model.collect_params()
params

standardrnn0_ (
  Parameter standardrnn0_hybridsequential0_embedding0_weight (shape=(33278, 200), dtype=float32)
  Parameter standardrnn0_lstm0_l0_i2h_weight (shape=(800, 200), dtype=<class 'numpy.float32'>)
  Parameter standardrnn0_lstm0_l0_h2h_weight (shape=(800, 200), dtype=<class 'numpy.float32'>)
  Parameter standardrnn0_lstm0_l0_i2h_bias (shape=(800,), dtype=<class 'numpy.float32'>)
  Parameter standardrnn0_lstm0_l0_h2h_bias (shape=(800,), dtype=<class 'numpy.float32'>)
  Parameter standardrnn0_lstm0_l1_i2h_weight (shape=(800, 200), dtype=<class 'numpy.float32'>)
  Parameter standardrnn0_lstm0_l1_h2h_weight (shape=(800, 200), dtype=<class 'numpy.float32'>)
  Parameter standardrnn0_lstm0_l1_i2h_bias (shape=(800,), dtype=<class 'numpy.float32'>)
  Parameter standardrnn0_lstm0_l1_h2h_bias (shape=(800,), dtype=<class 'numpy.float32'>)
  Parameter standardrnn0_hybridsequential0_embedding0_bias (shape=(33278,), dtype=float32)
)

In [33]:
W = params['standardrnn0_hybridsequential0_embedding0_weight'].data().asnumpy()
b = params['standardrnn0_hybridsequential0_embedding0_bias'].data().asnumpy()
W.shape, b.shape

((33278, 200), (33278,))

In [ ]:
%%time
U, Sigma, V = np.linalg.svd(W, full_matrices=True)

In [44]:
U.shape

(33278, 200)

In [29]:
np.random.normal(size=(100, 50)).shape

(100, 50)

In [6]:
def detach(hidden):
    if isinstance(hidden, (tuple, list)):
        hidden = [detach(i) for i in hidden]
    else:
        hidden = hidden.detach()
    return hidden

def evaluate(model, data_source, batch_size, ctx):
    total_L = 0.0
    ntotal = 0
    hidden = model.begin_state(
        batch_size=batch_size, func=mx.nd.zeros, ctx=ctx)
    for i, (data, target) in enumerate(data_source):
        data = data.as_in_context(ctx)
        target = target.as_in_context(ctx)
        output, hidden = model(data, hidden)
        hidden = detach(hidden)
        L = loss(output.reshape(-3, -1), target.reshape(-1))
        total_L += mx.nd.sum(L).asscalar()
        ntotal += L.size
    return total_L / ntotal

def train(model, train_data, val_data, test_data, epochs, lr):
    best_val = float("Inf")
    start_train_time = time.time()
    parameters = model.collect_params().values()
    for epoch in range(epochs):
        total_L = 0.0
        start_epoch_time = time.time()
        start_log_interval_time = time.time()
        hiddens = [model.begin_state(batch_size//len(context), func=mx.nd.zeros, ctx=ctx)
                   for ctx in context]
        for i, (data, target) in enumerate(train_data):
            data_list = gluon.utils.split_and_load(data, context,
                                                   batch_axis=1, even_split=True)
            target_list = gluon.utils.split_and_load(target, context,
                                                     batch_axis=1, even_split=True)
            hiddens = detach(hiddens)
            L = 0
            Ls = []
            with autograd.record():
                for j, (X, y, h) in enumerate(zip(data_list, target_list, hiddens)):
                    output, h = model(X, h)
                    batch_L = loss(output.reshape(-3, -1), y.reshape(-1,))
                    L = L + batch_L.as_in_context(context[0]) / (len(context) * X.size)
                    Ls.append(batch_L / (len(context) * X.size))
                    hiddens[j] = h
            L.backward()
            grads = [p.grad(x.context) for p in parameters for x in data_list]
            gluon.utils.clip_global_norm(grads, grad_clip)

            trainer.step(1)

            total_L += sum([mx.nd.sum(l).asscalar() for l in Ls])

            if i % log_interval == 0 and i > 0:
                cur_L = total_L / log_interval
                print('[Epoch %d Batch %d/%d] loss %.2f, ppl %.2f, '
                      'throughput %.2f samples/s'%(
                    epoch, i, len(train_data), cur_L, math.exp(cur_L),
                    batch_size * log_interval / (time.time() - start_log_interval_time)))
                total_L = 0.0
                start_log_interval_time = time.time()

        mx.nd.waitall()

        print('[Epoch %d] throughput %.2f samples/s'%(
                    epoch, len(train_data)*batch_size / (time.time() - start_epoch_time)))
        val_L = evaluate(model, val_data, batch_size, context[0])
        print('[Epoch %d] time cost %.2fs, valid loss %.2f, valid ppl %.2f'%(
            epoch, time.time()-start_epoch_time, val_L, math.exp(val_L)))

        if val_L < best_val:
            best_val = val_L
            test_L = evaluate(model, test_data, batch_size, context[0])
            model.save_parameters('{}_{}-{}.params'.format(model_name, dataset_name, epoch))
            print('test loss %.2f, test ppl %.2f'%(test_L, math.exp(test_L)))
        else:
            lr = lr*0.25
            print('Learning rate now %f'%(lr))
            trainer.set_learning_rate(lr)

    print('Total training throughput %.2f samples/s'%(
                            (batch_size * len(train_data) * epochs) /
                            (time.time() - start_train_time)))

In [7]:
train(model, train_data, val_data, test_data, epochs, lr)

[Epoch 0 Batch 200/2983] loss 7.66, ppl 2129.11, throughput 281.65 samples/s
[Epoch 0 Batch 400/2983] loss 6.75, ppl 853.92, throughput 286.11 samples/s
[Epoch 0 Batch 600/2983] loss 6.34, ppl 568.07, throughput 286.57 samples/s
[Epoch 0 Batch 800/2983] loss 6.18, ppl 483.26, throughput 286.67 samples/s
[Epoch 0 Batch 1000/2983] loss 6.05, ppl 422.96, throughput 286.33 samples/s
[Epoch 0 Batch 1200/2983] loss 5.97, ppl 391.82, throughput 286.72 samples/s
[Epoch 0 Batch 1400/2983] loss 5.87, ppl 352.51, throughput 286.17 samples/s
[Epoch 0 Batch 1600/2983] loss 5.86, ppl 351.89, throughput 286.47 samples/s
[Epoch 0 Batch 1800/2983] loss 5.70, ppl 299.67, throughput 286.51 samples/s
[Epoch 0 Batch 2000/2983] loss 5.67, ppl 289.43, throughput 286.45 samples/s
[Epoch 0 Batch 2200/2983] loss 5.57, ppl 261.92, throughput 283.67 samples/s
[Epoch 0 Batch 2400/2983] loss 5.58, ppl 264.64, throughput 287.07 samples/s
[Epoch 0 Batch 2600/2983] loss 5.56, ppl 259.61, throughput 286.79 samples/s
[E